In [1]:
! pip install "paddlex<=1.3.11" -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.0/517.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4

[notice] A new release of pip available: 22.1.2 -> 23.3.2
[notice] To update, run: pip install --upgra

## 2. 准备昆虫目标检测数据集

In [2]:
! wget https://bj.bcebos.com/paddlex/datasets/insect_det.tar.gz
! tar xzf insect_det.tar.gz

--2024-01-13 20:02:41--  https://bj.bcebos.com/paddlex/datasets/insect_det.tar.gz
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17304082 (17M) [application/octet-stream]
Saving to: ‘insect_det.tar.gz.1’

insect_det.tar.gz.1 100%[===================>]  16.50M  92.6MB/s    in 0.2s    

2024-01-13 20:02:41 (92.6 MB/s) - ‘insect_det.tar.gz.1’ saved [17304082/17304082]


## 3. 模型训练

### 3.1 配置GPU

In [3]:
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

### 3.2 定义图像处理流程transforms
定义数据处理流程，其中训练和测试需分别定义，训练过程包括了部分测试过程中不需要的数据增强操作，如在本示例中，训练过程使用了`RandomHorizontalFlip`数据增强方式，更多图像预处理流程transforms的使用可参见[paddlex.det.transforms](https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/det_transforms.html)。

In [4]:
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32),
])

### 3.3 定义数据集Dataset
目标检测可使用`VOCDetection`格式和`COCODetection`两种数据集，此处由于数据集为VOC格式，因此采用`pdx.datasets.VOCDetection`来加载数据集，该接口的介绍可参见文档[paddlex.datasets.VOCDetection](https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-vocdetection)。

In [5]:
train_dataset = pdx.datasets.VOCDetection(
    data_dir='insect_det',
    file_list='insect_det/train_list.txt',
    label_list='insect_det/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='insect_det',
    file_list='insect_det/val_list.txt',
    label_list='insect_det/labels.txt',
    transforms=eval_transforms)

2024-01-13 20:02:44 [INFO]	Starting to read file list from dataset...
2024-01-13 20:02:45 [INFO]	169 samples in file insect_det/train_list.txt
creating index...
index created!
2024-01-13 20:02:45 [INFO]	Starting to read file list from dataset...
2024-01-13 20:02:45 [INFO]	24 samples in file insect_det/val_list.txt
creating index...
index created!


### 3.4 模型开始训练
使用本数据集在P40上训练，如有GPU，模型的训练过程预估为9分钟左右；如无GPU，则预估为6小时左右。更多训练模型的参数可参见文档[paddlex.det.FasterRCNN](https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#paddlex-det-fasterrcnn)。模型训练过程每间隔save_interval_epochs轮会保存一次模型在save_dir目录下，同时在保存的过程中也会在验证数据集上计算相关指标，具体相关日志参见[文档](https://paddlex.readthedocs.io/zh_CN/develop/appendix/metrics.html#fasterrcnn)。  

**AIStudio使用VisualDL查看训练过程中的指标变化**  
1. 点击左边菜单图标的『**可视化**』；  
2. 设置logdir，logdir的路径为训练代码中`save_dir`指定的目录下的vdl_log目录，例如`output/faster_rcnn_r50_fpn/vdl_log`  
3. 点击下方『**启动VisualDL服务按钮**』，再『**打开VisualDL**』即可

In [6]:
num_classes = len(train_dataset.labels) + 1
model = pdx.det.FasterRCNN(num_classes=num_classes)
model.train(
    num_epochs=12,
    train_dataset=train_dataset,
    train_batch_size=2,
    eval_dataset=eval_dataset,
    learning_rate=0.0025,
    lr_decay_epochs=[8, 11],
    save_interval_epochs=1,
    save_dir='output/faster_rcnn_r50_fpn',
    use_vdl=True)

2024-01-13 20:02:46,386-INFO: If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!


2024-01-13 20:02:47 [INFO]	Downloading ResNet50_cos_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_cos_pretrained.tar


100%|██████████| 100310/100310 [00:01<00:00, 55497.64KB/s]


2024-01-13 20:02:49 [INFO]	Decompressing output/faster_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.tar...


W0113 20:02:49.549751    98 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 12.0, Runtime API Version: 9.0
W0113 20:02:49.614439    98 device_context.cc:260] device: 0, cuDNN Version: 7.6.


...

2024-01-13 20:09:07 [INFO]	[EVAL] Finished, Epoch=10, bbox_map=80.001776 .
2024-01-13 20:09:12 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_10.
2024-01-13 20:09:12 [INFO]	Current evaluated best model in eval_dataset is epoch_9, bbox_map=80.70238705781438
2024-01-13 20:09:19 [INFO]	[TRAIN] Epoch=11/12, Step=2/84, loss=0.293816, loss_cls=0.196824, loss_bbox=0.085477, loss_rpn_cls=0.002718, loss_rpn_bbox=0.008797, lr=0.00025, time_each_step=0.48s, eta=0:1:22
2024-01-13 20:09:20 [INFO]	[TRAIN] Epoch=11/12, Step=4/84, loss=0.29466, loss_cls=0.197164, loss_bbox=0.079928, loss_rpn_cls=0.006112, loss_rpn_bbox=0.011457, lr=0.00025, time_each_step=0.51s, eta=0:1:24
2024-01-13 20:09:21 [INFO]	[TRAIN] Epoch=11/12, Step=6/84, loss=0.366855, loss_cls=0.253677, loss_bbox=0.083519, loss_rpn_cls=0.013089, loss_rpn_bbox=0.016571, lr=0.00025, time_each_step=0.54s, eta=0:1:25
2024-01-13 20:09:21 [INFO]	[TRAIN] Epoch=11/12, Step=8/84, loss=0.298055, loss_cls=0.20263, loss_bbox=0.082768, loss_rpn_

100%|██████████| 24/24 [00:06<00:00,  3.80it/s]


2024-01-13 20:09:40 [INFO]	[EVAL] Finished, Epoch=11, bbox_map=82.276537 .
2024-01-13 20:09:49 [INFO]	Model saved in output/faster_rcnn_r50_fpn/best_model.
2024-01-13 20:09:53 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_11.
2024-01-13 20:09:53 [INFO]	Current evaluated best model in eval_dataset is epoch_11, bbox_map=82.27653734560329
2024-01-13 20:10:00 [INFO]	[TRAIN] Epoch=12/12, Step=2/84, loss=0.377876, loss_cls=0.240915, loss_bbox=0.123306, loss_rpn_cls=0.005754, loss_rpn_bbox=0.007902, lr=2.5e-05, time_each_step=0.48s, eta=0:0:58
2024-01-13 20:10:01 [INFO]	[TRAIN] Epoch=12/12, Step=4/84, loss=0.416052, loss_cls=0.323605, loss_bbox=0.069295, loss_rpn_cls=0.003067, loss_rpn_bbox=0.020085, lr=2.5e-05, time_each_step=0.52s, eta=0:1:0
2024-01-13 20:10:02 [INFO]	[TRAIN] Epoch=12/12, Step=6/84, loss=0.186789, loss_cls=0.128688, loss_bbox=0.050589, loss_rpn_cls=0.002232, loss_rpn_bbox=0.00528, lr=2.5e-05, time_each_step=0.54s, eta=0:1:1
2024-01-13 20:10:03 [INFO]	[TRAIN] Epoch=

100%|██████████| 24/24 [00:05<00:00,  4.13it/s]


2024-01-13 20:10:20 [INFO]	[EVAL] Finished, Epoch=12, bbox_map=81.102318 .
2024-01-13 20:10:25 [INFO]	Model saved in output/faster_rcnn_r50_fpn/epoch_12.
2024-01-13 20:10:25 [INFO]	Current evaluated best model in eval_dataset is epoch_11, bbox_map=82.27653734560329


## 4 模型预测

使用模型进行预测，同时使用`pdx.det.visualize`将结果可视化，可视化结果将保存到`./output/faster_rcnn_r50_fpn`下，其中`threshold`代表Box的置信度阈值，将Box置信度低于该阈值的框过滤不进行可视化。
> 此处请在AIStudio Notebook页面的右上角菜单，选择重启执行器，以释放显存，重新加载训练好的模型

In [7]:
import paddlex as pdx
model = pdx.load_model('output/faster_rcnn_r50_fpn/best_model')
image_name = 'insect_det/JPEGImages/0216.jpg'
result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5, save_dir='./output/faster_rcnn_r50_fpn')

2024-01-13 20:10:27 [INFO]	Model[FasterRCNN] loaded.


2024-01-13 20:10:28,120-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2024-01-13 20:10:28,888-INFO: generated new fontManager


2024-01-13 20:10:29 [INFO]	The visualized result is saved as ./output/faster_rcnn_r50_fpn/visualize_0216.jpg


可视化结果如下所示：  
<img src="https://ai-studio-static-online.cdn.bcebos.com/a6b37f6ab44443e19bef8390f6c9a1038541ed68644740a6b5a5a01a4ee5738d" width="50%" height="50%">

## 5. 反馈
PaddleX用户QQ群: 1045148026 (手机QQ扫描如下二维码快速加入)  
<img src="https://ai-studio-static-online.cdn.bcebos.com/35a3a4c0748f4eb393b3acb100bc14f2e9920eed74244c1bbc2041bdd6790f81" width="15%" height="15%">